In [1]:
import boto3
import json
from pathlib import Path

## Class

In [2]:
class IotThing():
    def __init__(self, thing_name:str, key_path:Path, default_policy:str, default_group:str, profile_name='default', verbose=True):
        self.thing_name = thing_name
        self.key_path = key_path
        self.default_policy = default_policy
        self.default_group = default_group
        if not isinstance(self.key_path, Path):
            print(f"Keypath must be type Path, but is type: {type(self.key_path)}")
            print(f"Trying to convert to Path")
            try:
                self.key_path = Path(self.key_path)
                print(f"Converted to Path")
            except:
                print(f"Could not convert to Path")
                raise TypeError(f"Keypath must be type Path, but is type: {type(self.key_path)}")
        self.thingArn = ''
        self.thingId = ''
        self.certificateArn = ''
        self.certificateId = ''
        self.session = boto3.setup_default_session(profile_name=profile_name)
        self.thing_client = boto3.client('iot')
        self.create_thing()
        self.create_keys_and_certificate()
        self.attach_policy()
        self.attach_thing_principal()
        self.attach_thing_to_group()
        self.close()
        if verbose:
            print(f"{self.thing_name} created.")
        
    def close(self):
        self.thing_client.close()
        
    def attach_client(self):
        self.thing_client = boto3.client('iot')
        
    def create_thing(self):
        thingResponse = self.thing_client.create_thing(thingName=self.thing_name)
        data = json.loads(json.dumps(thingResponse, sort_keys=False, indent=4))
        for element in data: 
            if element == 'thingArn':
                self.thingArn = data['thingArn']
            elif element == 'thingId':
                self.thingId = data['thingId']
                
    def list_things(self):
        return self.thing_client.list_things()
    
    def create_keys_and_certificate(self):
        certResponse = self.thing_client.create_keys_and_certificate(setAsActive=True)
        data = json.loads(json.dumps(certResponse, sort_keys=False, indent=4))
        # Create folder for keys and certificates
        (self.key_path/self.thing_name).mkdir(parents=True, exist_ok=False)
        for element in data: 
            if element == 'certificateArn':
                self.certificateArn = data['certificateArn']
            elif element == 'keyPair':
                PublicKey = data['keyPair']['PublicKey']
                PrivateKey = data['keyPair']['PrivateKey']
            elif element == 'certificatePem':
                certificatePem = data['certificatePem']
            elif element == 'certificateId':
                self.certificateId = data['certificateId']
                					
        with open(self.key_path/self.thing_name/f'{self.thing_name}-public.key', 'w') as outfile:
                outfile.write(PublicKey)
        with open(self.key_path/self.thing_name/f'{self.thing_name}-private.key', 'w') as outfile:
                outfile.write(PrivateKey)
        with open(self.key_path/self.thing_name/f'{self.thing_name}-cert.pem', 'w') as outfile:
                outfile.write(certificatePem)
                
    def attach_policy(self):
        self.thing_client.attach_policy(
            policyName = self.default_policy,
            target = self.certificateArn
        )
        
    def attach_thing_principal(self):
        self.thing_client.attach_thing_principal(
            thingName = self.thing_name,
            principal = self.certificateArn
        )
        
    def attach_thing_to_group(self):
        self.thing_client.add_thing_to_thing_group(
            thingGroupName = self.default_group,
            thingGroupArn = self.get_group_arn(),
            thingName = self.thing_name,
            thingArn = self.thingArn
        )
        
    def get_group_arn(self):
        group_list = self.thing_client.list_thing_groups()
        group_arn = ''
        group_not_found = True
        data = json.loads(json.dumps(group_list, sort_keys=False, indent=4))
        for element in data:
            if element == 'thingGroups':
                for group in group_list['thingGroups']:
                    if group['groupName'] == self.default_group:
                        group_not_found = False
                        group_arn = group['groupArn']
        if group_not_found:
            print(f"Group {self.default_group} not found")
        return group_arn
    
    def close(self):
        self.thing_client.close()
    
    # destructor IotThing
    def __del__(self):
        try:
            print(f"Deleting thing {self.thing_name}")
            self.thing_client.detach_thing_principal(
                thingName = self.thing_name,
                principal = self.certificateArn
            )
            self.thing_client.detach_policy(
                policyName = self.default_policy,
                target = self.certificateArn
            )
            self.thing_client.update_certificate(
                certificateId = self.certificateId, 
                newStatus='INACTIVE'
            )
            self.thing_client.delete_certificate(
                certificateId = self.certificateId,
                forceDelete = True
            )
            self.thing_client.remove_thing_from_thing_group(
                thingGroupName = self.default_group,
                thingGroupArn = self.get_group_arn(),
                thingName = self.thing_name,
                thingArn = self.thingArn
            )
            self.thing_client.delete_thing(
                thingName = self.thing_name
            )
        except Exception as e:
            print(e)
        self.close()
        # delete folder with keys and certificates
        [fn.unlink(missing_ok=True) for fn in list((self.key_path/self.thing_name).iterdir())]
        (self.key_path/self.thing_name).rmdir()

## Create Things

In [3]:
i = 0
n_things = 5
thing_name = 'car-iot-thing'
thing_group = 'Car_Group' #'The_Coolest_Things' #'auto-iot-group'
thing_policy = 'p_Car_Group' #'p_Test_Thing' #'auto-iot-policy'
key_path = Path('/media/netzdose/constantin/Uni/UIllinois/courses/CS437_IoT/labs/lab_04/certificates/')

In [4]:
### Create things
l_iot_things = []
for i in range(n_things):
    l_iot_things.append(
        IotThing(f"{thing_name}_{i:04d}",
                        key_path=key_path,
                        default_policy=thing_policy,
                        default_group=thing_group, 
                        )
    )


car-iot-thing_0000 created.
car-iot-thing_0001 created.
car-iot-thing_0002 created.
car-iot-thing_0003 created.
car-iot-thing_0004 created.


In [5]:
### Delete Things
for thing in l_iot_things:
    thing.__del__()
del l_iot_things

Deleting thing car-iot-thing_0000
Deleting thing car-iot-thing_0001
Deleting thing car-iot-thing_0002
Deleting thing car-iot-thing_0003
Deleting thing car-iot-thing_0004


Exception ignored in: <function IotThing.__del__ at 0x7f384e240790>
Traceback (most recent call last):
  File "/tmp/ipykernel_74937/1938956604.py", line 143, in __del__
  File "/media/sda/miniconda3/envs/sci-vis/lib/python3.10/pathlib.py", line 1015, in iterdir
    for name in self._accessor.listdir(self):
FileNotFoundError: [Errno 2] No such file or directory: '/media/netzdose/constantin/Uni/UIllinois/courses/CS437_IoT/labs/lab_04/certificates/car-iot-thing_0003'


Deleting thing car-iot-thing_0003
An error occurred (ResourceNotFoundException) when calling the DetachThingPrincipal operation: Can't detach the principal arn:aws:iot:eu-central-1:524422720739:cert/f42fc66d1694c8b54c9b9be4af4e59326a6ae7df8e63bc083fd7c3b953bb4e35 from thing car-iot-thing_0003 because the principal does not exist in your account
Deleting thing car-iot-thing_0002


Exception ignored in: <function IotThing.__del__ at 0x7f384e240790>
Traceback (most recent call last):
  File "/tmp/ipykernel_74937/1938956604.py", line 143, in __del__
  File "/media/sda/miniconda3/envs/sci-vis/lib/python3.10/pathlib.py", line 1015, in iterdir
    for name in self._accessor.listdir(self):
FileNotFoundError: [Errno 2] No such file or directory: '/media/netzdose/constantin/Uni/UIllinois/courses/CS437_IoT/labs/lab_04/certificates/car-iot-thing_0002'
Exception ignored in: <function IotThing.__del__ at 0x7f384e240790>
Traceback (most recent call last):
  File "/tmp/ipykernel_74937/1938956604.py", line 143, in __del__
  File "/media/sda/miniconda3/envs/sci-vis/lib/python3.10/pathlib.py", line 1015, in iterdir
    for name in self._accessor.listdir(self):
FileNotFoundError: [Errno 2] No such file or directory: '/media/netzdose/constantin/Uni/UIllinois/courses/CS437_IoT/labs/lab_04/certificates/car-iot-thing_0001'


An error occurred (ResourceNotFoundException) when calling the DetachThingPrincipal operation: Can't detach the principal arn:aws:iot:eu-central-1:524422720739:cert/be50930e2481021ebc7505ecfe96f0e45d85407b0bb05e4fb99e144d0bdbe3b0 from thing car-iot-thing_0002 because the principal does not exist in your account
Deleting thing car-iot-thing_0001
An error occurred (ResourceNotFoundException) when calling the DetachThingPrincipal operation: Can't detach the principal arn:aws:iot:eu-central-1:524422720739:cert/d62acbbc6f92e87e6e92f1fda1ac4a85baf9c95cb5927aa2ee0fbf470fdb3735 from thing car-iot-thing_0001 because the principal does not exist in your account
Deleting thing car-iot-thing_0000


Exception ignored in: <function IotThing.__del__ at 0x7f384e240790>
Traceback (most recent call last):
  File "/tmp/ipykernel_74937/1938956604.py", line 143, in __del__
  File "/media/sda/miniconda3/envs/sci-vis/lib/python3.10/pathlib.py", line 1015, in iterdir
    for name in self._accessor.listdir(self):
FileNotFoundError: [Errno 2] No such file or directory: '/media/netzdose/constantin/Uni/UIllinois/courses/CS437_IoT/labs/lab_04/certificates/car-iot-thing_0000'


An error occurred (ResourceNotFoundException) when calling the DetachThingPrincipal operation: Can't detach the principal arn:aws:iot:eu-central-1:524422720739:cert/265c5022f662175446b7816508abe02c8e2f1b355ce342a2c5f30b4a807c1844 from thing car-iot-thing_0000 because the principal does not exist in your account


## Unit Tests

In [64]:
# Unit tests for IotThing class

thing_name = 'auto-iot-thing'
thing_group = 'The_Coolest_Things' #'auto-iot-group'
thing_policy = 'p_Test_Thing' #'auto-iot-policy'
key_path = Path('/media/netzdose/constantin/Uni/UIllinois/courses/CS437_IoT/labs/lab_04/certificates/')

# Test 1: Create a thing
i = 10
new_thing = IotThing(f"{thing_name}_{i:04d}",
         key_path=key_path,
         default_policy=thing_policy,
         default_group=thing_group,         
         )
assert isinstance(new_thing, IotThing)
del new_thing

# Test 1.5: Delete a thing
del new_thing

# Test 2: Create a thing with a name that already exists

# Test 3: Create a thing and could not find group

# Test 4: Create a thing and key path already exists

Exception ignored in: <function IotThing.__del__ at 0x7f2335acacb0>
Traceback (most recent call last):
  File "/tmp/ipykernel_10223/507940076.py", line 118, in __del__
  File "/media/sda/miniconda3/envs/sci-vis/lib/python3.10/site-packages/botocore/client.py", line 530, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/media/sda/miniconda3/envs/sci-vis/lib/python3.10/site-packages/botocore/client.py", line 960, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.CertificateStateException: An error occurred (CertificateStateException) when calling the DeleteCertificate operation: Certificate must be deactivated (not ACTIVE) before deletion. Id: 07a829290cf10beb1c1225f42417c117684c44f1cfb932c04d45dee9b50c33ff


Deleting thing auto-iot-thing_0010


## Misc

### Inactivate Certificate

In [95]:
iot_thing.thing_client.update_certificate(certificateId='ecb75cf3a4c2113f62638c0980441a240be5f060b797a17e28c7797dd5a72b05', 
                                          newStatus='INACTIVE'
                                          )

NameError: name 'iot_thing' is not defined

### Single connection

In [ ]:
thing_client = boto3.client('iot')

In [ ]:
thing_client.list_things()

{'ResponseMetadata': {'RequestId': '59e96aa7-8aa6-4c9a-9dc8-e2db1630725e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 26 Mar 2023 20:25:01 GMT',
   'content-type': 'application/json',
   'content-length': '497',
   'connection': 'keep-alive',
   'x-amzn-requestid': '59e96aa7-8aa6-4c9a-9dc8-e2db1630725e'},
  'RetryAttempts': 0},
 'things': [{'thingName': 'auto-iot-thing_0000',
   'thingArn': 'arn:aws:iot:eu-central-1:524422720739:thing/auto-iot-thing_0000',
   'attributes': {},
   'version': 1},
  {'thingName': 'auto-iot-thing_0001',
   'thingArn': 'arn:aws:iot:eu-central-1:524422720739:thing/auto-iot-thing_0001',
   'attributes': {},
   'version': 1},
  {'thingName': 'Test_Thing',
   'thingArn': 'arn:aws:iot:eu-central-1:524422720739:thing/Test_Thing',
   'attributes': {},
   'version': 1}]}